In [ ]:
"""
GPU-Accelerated Object Detection Pipeline - Phase 1
Setup and Core Implementation for NVIDIA DL Project
"""

# ============================================================================
# PART 1: ENVIRONMENT SETUP & VERIFICATION
# ============================================================================

import torch
import torchvision
import sys
import os
from pathlib import Path

print("="*70)
print("ENVIRONMENT VERIFICATION")
print("="*70)

# Check Python version
print(f"Python version: {sys.version}")

# Check PyTorch and CUDA availability
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  WARNING: CUDA not available. Using CPU.")

print("="*70)

ENVIRONMENT VERIFICATION
Python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
PyTorch version: 2.8.0+cu126
Torchvision version: 0.23.0+cu126
CUDA available: True
CUDA version: 12.6
GPU Device: Tesla T4
GPU Memory: 15.83 GB


In [ ]:
# ============================================================================
# PART 2: INSTALL DEPENDENCIES
# ============================================================================

print("\nInstalling required packages...")
# Uncomment in Colab:
# !pip install -q opencv-python-headless
# !pip install -q albumentations
# !pip install -q pycocotools
# !pip install -q tensorboard

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json
import time
from datetime import datetime

print("✓ All packages imported successfully!")



Installing required packages...
✓ All packages imported successfully!


In [ ]:
# ============================================================================
# PART 3: PROJECT STRUCTURE SETUP
# ============================================================================

# Create project directories
project_dirs = [
    'data/raw',
    'data/processed',
    'data/annotations',
    'models/checkpoints',
    'models/pretrained',
    'outputs/images',
    'outputs/videos',
    'outputs/metrics',
    'logs',
    'notebooks'
]

for dir_path in project_dirs:
    Path(dir_path).mkdir(parents=True, exist_ok=True)

print(f"✓ Created {len(project_dirs)} project directories")


✓ Created 10 project directories


In [ ]:
# ============================================================================
# PART 4: DATASET PREPARATION - COCO Dataset
# ============================================================================

print("\n" + "="*70)
print("DATASET SETUP - COCO 2017")
print("="*70)


def download_coco_subset():
    """Download COCO validation dataset (1GB vs 18GB for training)"""
    import urllib.request
    import zipfile

    base_url = "http://images.cocodataset.org/zips/"
    anno_url = "http://images.cocodataset.org/annotations/"

    files = {
        'val2017.zip': base_url,
        'annotations_trainval2017.zip': anno_url
    }

    for filename, url in files.items():
        filepath = f'data/{filename}'
        if not os.path.exists(filepath):
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(url + filename, filepath)
            print(f"✓ Downloaded {filename}")

            # Extract
            print(f"Extracting {filename}...")
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                zip_ref.extractall('data/')
            print(f"✓ Extracted {filename}")
        else:
            print(f"✓ {filename} already exists")

download_coco_subset()




DATASET SETUP - COCO 2017
✓ Downloaded val2017.zip
Extracting val2017.zip...
✓ Extracted val2017.zip
✓ Downloaded annotations_trainval2017.zip
Extracting annotations_trainval2017.zip...
✓ Extracted annotations_trainval2017.zip


In [ ]:
# ============================================================================
# PART 5: CUSTOM DATASET CLASS
# ============================================================================

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class CustomObjectDetectionDataset(Dataset):
    """
    Custom dataset for object detection
    Compatible with COCO format annotations
    """
    def __init__(self, image_dir, annotation_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform

        # Load annotations
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)

        self.images = self.coco_data['images']
        self.annotations = self.coco_data['annotations']

        # Create image_id to annotations mapping
        self.img_to_anns = {}
        for ann in self.annotations:
            img_id = ann['image_id']
            if img_id not in self.img_to_anns:
                self.img_to_anns[img_id] = []
            self.img_to_anns[img_id].append(ann)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Get image info
        img_info = self.images[idx]
        img_path = os.path.join(self.image_dir, img_info['file_name'])

        # Load image
        image = Image.open(img_path).convert('RGB')

        # Get annotations for this image
        img_id = img_info['id']
        anns = self.img_to_anns.get(img_id, [])

        # Extract boxes and labels
        boxes = []
        labels = []
        for ann in anns:
            # COCO format: [x, y, width, height]
            x, y, w, h = ann['bbox']
            # Convert to [x_min, y_min, x_max, y_max]
            boxes.append([x, y, x + w, y + h])
            labels.append(ann['category_id'])

        # Convert to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id])
        }

        if self.transform:
            image = self.transform(image)

        return image, target

In [ ]:
# ============================================================================
# PART 6: DATA TRANSFORMS & AUGMENTATION
# ============================================================================

def get_transforms(train=True):
    """Define data transforms for training and validation"""
    if train:
        return transforms.Compose([
            transforms.Resize((640, 640)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((640, 640)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])


In [ ]:
# ============================================================================
# PART 7: MODEL SETUP - FASTER R-CNN WITH RESNET50 BACKBONE
# ============================================================================

import torchvision.models.detection as detection

def create_model(num_classes=91, pretrained=True):
    """
    Create Faster R-CNN model with ResNet50 backbone
    num_classes: 91 for COCO (90 objects + background)
    """
    # Load pretrained Faster R-CNN
    model = detection.fasterrcnn_resnet50_fpn(pretrained=pretrained)

    # Replace the classifier head for custom number of classes
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = detection.faster_rcnn.FastRCNNPredictor(
        in_features, num_classes
    )

    return model

In [ ]:
# ============================================================================
# PART 8: TRAINING CONFIGURATION
# ============================================================================

class TrainingConfig:
    """Configuration for training pipeline"""
    # Dataset
    TRAIN_IMAGE_DIR = 'data/val2017'  # Using val set for quick training
    TRAIN_ANNO_FILE = 'data/annotations/instances_val2017.json'

    # Training hyperparameters
    BATCH_SIZE = 4
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.005
    MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0005

    # Model
    NUM_CLASSES = 91  # COCO has 90 classes + background

    # Device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Checkpointing
    CHECKPOINT_DIR = 'models/checkpoints'
    SAVE_FREQUENCY = 2  # Save every N epochs

    # Logging
    LOG_DIR = 'logs'
    PRINT_FREQUENCY = 10  # Print every N batches

config = TrainingConfig()

print("\n" + "="*70)
print("TRAINING CONFIGURATION")
print("="*70)
print(f"Device: {config.DEVICE}")
print(f"Batch Size: {config.BATCH_SIZE}")
print(f"Learning Rate: {config.LEARNING_RATE}")
print(f"Epochs: {config.NUM_EPOCHS}")
print(f"Number of Classes: {config.NUM_CLASSES}")
print("="*70)


TRAINING CONFIGURATION
Device: cuda
Batch Size: 4
Learning Rate: 0.005
Epochs: 10
Number of Classes: 91


In [ ]:
# ============================================================================
# PART 9: TRAINING UTILITIES
# ============================================================================

class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def save_checkpoint(model, optimizer, epoch, loss, filename):
    """Save model checkpoint"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'timestamp': datetime.now().isoformat()
    }
    torch.save(checkpoint, filename)
    print(f"✓ Checkpoint saved: {filename}")

def load_checkpoint(model, optimizer, filename):
    """Load model checkpoint"""
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"✓ Checkpoint loaded from epoch {epoch}")
    return epoch, loss

In [ ]:
# ============================================================================
# PART 10: QUICK TEST WITH DUMMY DATA
# ============================================================================

print("\n" + "="*70)
print("QUICK MODEL TEST")
print("="*70)

# Create model
model = create_model(num_classes=config.NUM_CLASSES, pretrained=True)
model = model.to(config.DEVICE)
model.eval()

# Create dummy input
dummy_image = torch.rand(1, 3, 640, 640).to(config.DEVICE)

print("Running inference on dummy image...")
with torch.no_grad():
    output = model(dummy_image)

print(f"✓ Model inference successful!")
print(f"  Output keys: {output[0].keys()}")
print(f"  Number of detections: {len(output[0]['boxes'])}")

# Calculate model parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel Statistics:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size: ~{total_params * 4 / 1e6:.2f} MB (float32)")

print("\n" + "="*70)
print("✓ PHASE 1 SETUP COMPLETE!")
print("="*70)
print("\nNext steps:")
print("1. Download COCO dataset (uncomment download_coco_subset())")
print("2. Run the training script (coming in next artifact)")
print("3. Monitor training metrics")
print("="*70)


QUICK MODEL TEST


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:01<00:00, 133MB/s]


Running inference on dummy image...
✓ Model inference successful!
  Output keys: dict_keys(['boxes', 'labels', 'scores'])
  Number of detections: 0

Model Statistics:
  Total parameters: 41,755,286
  Trainable parameters: 41,532,886
  Model size: ~167.02 MB (float32)

✓ PHASE 1 SETUP COMPLETE!

Next steps:
1. Download COCO dataset (uncomment download_coco_subset())
2. Run the training script (coming in next artifact)
3. Monitor training metrics


In [ ]:
"""
Training Script for GPU-Accelerated Object Detection
Implements training loop with metrics tracking and checkpointing
"""

import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from tqdm import tqdm
import json
from datetime import datetime

# ============================================================================
# TRAINING FUNCTION
# ============================================================================

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10):
    """
    Train for one epoch

    Args:
        model: The detection model
        optimizer: Optimizer
        data_loader: Training data loader
        device: Device to train on
        epoch: Current epoch number
        print_freq: Print frequency

    Returns:
        Average loss for the epoch
    """
    model.train()

    loss_meter = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()

    end = time.time()

    # Progress bar
    pbar = tqdm(data_loader, desc=f'Epoch {epoch}')

    for batch_idx, (images, targets) in enumerate(pbar):
        # Measure data loading time
        data_time.update(time.time() - end)

        # Move to device
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        # Update metrics
        loss_meter.update(losses.item(), len(images))
        batch_time.update(time.time() - end)
        end = time.time()

        # Update progress bar
        pbar.set_postfix({
            'loss': f'{loss_meter.avg:.4f}',
            'batch_time': f'{batch_time.avg:.3f}s',
            'data_time': f'{data_time.avg:.3f}s'
        })

        # Print detailed stats periodically
        if (batch_idx + 1) % print_freq == 0:
            print(f'\nEpoch: [{epoch}][{batch_idx + 1}/{len(data_loader)}]  '
                  f'Loss: {loss_meter.avg:.4f}  '
                  f'Batch Time: {batch_time.avg:.3f}s  '
                  f'Data Time: {data_time.avg:.3f}s')

            # Print individual losses
            for loss_name, loss_value in loss_dict.items():
                print(f'  {loss_name}: {loss_value.item():.4f}')

    return loss_meter.avg

In [ ]:
# ============================================================================
# VALIDATION FUNCTION
# ============================================================================

@torch.no_grad()
def validate(model, data_loader, device):
    """
    Validate the model

    Args:
        model: The detection model
        data_loader: Validation data loader
        device: Device to validate on

    Returns:
        Validation metrics
    """
    model.eval()

    loss_meter = AverageMeter()
    inference_time = AverageMeter()

    pbar = tqdm(data_loader, desc='Validation')

    for images, targets in pbar:
        # Move to device
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Measure inference time
        start_time = time.time()

        # For validation, we need to set model to train mode to get losses
        model.train()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        model.eval()

        inference_time.update(time.time() - start_time, len(images))
        loss_meter.update(losses.item(), len(images))

        pbar.set_postfix({
            'loss': f'{loss_meter.avg:.4f}',
            'inference_time': f'{inference_time.avg:.3f}s'
        })

    return {
        'loss': loss_meter.avg,
        'inference_time': inference_time.avg
    }


In [ ]:
# ============================================================================
# MAIN TRAINING LOOP
# ============================================================================

def train_model(model, train_loader, val_loader, config):
    """
    Main training loop

    Args:
        model: The detection model
        train_loader: Training data loader
        val_loader: Validation data loader
        config: Training configuration
    """
    # Setup optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.SGD(
        params,
        lr=config.LEARNING_RATE,
        momentum=config.MOMENTUM,
        weight_decay=config.WEIGHT_DECAY
    )

    # Learning rate scheduler
    lr_scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=3,
        gamma=0.1
    )

    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_inference_time': [],
        'learning_rate': []
    }

    print("\n" + "="*70)
    print("STARTING TRAINING")
    print("="*70)

    best_val_loss = float('inf')
    start_time = time.time()

    for epoch in range(1, config.NUM_EPOCHS + 1):
        epoch_start_time = time.time()

        print(f"\n{'='*70}")
        print(f"Epoch {epoch}/{config.NUM_EPOCHS}")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        print(f"{'='*70}")

        # Training
        train_loss = train_one_epoch(
            model, optimizer, train_loader,
            config.DEVICE, epoch, config.PRINT_FREQUENCY
        )

        # Validation
        print("\nRunning validation...")
        val_metrics = validate(model, val_loader, config.DEVICE)

        # Update learning rate
        lr_scheduler.step()

        # Record history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_metrics['loss'])
        history['val_inference_time'].append(val_metrics['inference_time'])
        history['learning_rate'].append(optimizer.param_groups[0]['lr'])

        # Print epoch summary
        epoch_time = time.time() - epoch_start_time
        print(f"\n{'='*70}")
        print(f"Epoch {epoch} Summary:")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_metrics['loss']:.4f}")
        print(f"  Val Inference Time: {val_metrics['inference_time']:.3f}s")
        print(f"  Epoch Time: {epoch_time:.2f}s")
        print(f"{'='*70}")

        # Save checkpoint
        if epoch % config.SAVE_FREQUENCY == 0:
            checkpoint_path = f"{config.CHECKPOINT_DIR}/checkpoint_epoch_{epoch}.pth"
            save_checkpoint(model, optimizer, epoch, train_loss, checkpoint_path)

        # Save best model
        if val_metrics['loss'] < best_val_loss:
            best_val_loss = val_metrics['loss']
            best_model_path = f"{config.CHECKPOINT_DIR}/best_model.pth"
            save_checkpoint(model, optimizer, epoch, val_metrics['loss'], best_model_path)
            print(f"✓ New best model saved! Val Loss: {best_val_loss:.4f}")

    # Training complete
    total_time = time.time() - start_time
    print(f"\n{'='*70}")
    print("TRAINING COMPLETE!")
    print(f"Total Training Time: {total_time/3600:.2f} hours")
    print(f"Best Validation Loss: {best_val_loss:.4f}")
    print(f"{'='*70}")

    # Save training history
    history_path = f"{config.LOG_DIR}/training_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(history_path, 'w') as f:
        json.dump(history, f, indent=4)
    print(f"✓ Training history saved: {history_path}")

    return history

In [ ]:

def test_dataloader():
    """Test the data loader with a few samples"""
    print("\n" + "="*70)
    print("TESTING DATA LOADER")
    print("="*70)

    # Create a minimal test dataset
    # In practice, you'd use the COCO dataset

    print("Creating data loaders...")

    # For testing, we'll create dummy data loaders
    # Replace this with actual COCO dataset when ready

    class DummyDataset(torch.utils.data.Dataset):
        def __init__(self, num_samples=100):
            self.num_samples = num_samples

        def __len__(self):
            return self.num_samples

        def __getitem__(self, idx):
            # Dummy image
            image = torch.rand(3, 640, 640)

            # Dummy target
            num_boxes = torch.randint(1, 10, (1,)).item()
            boxes = torch.rand(num_boxes, 4) * 640
            boxes[:, 2:] += boxes[:, :2]  # Ensure x2, y2 > x1, y1

            target = {
                'boxes': boxes,
                'labels': torch.randint(1, 91, (num_boxes,)),
                'image_id': torch.tensor([idx])
            }

            return image, target

    # Create datasets
    train_dataset = DummyDataset(num_samples=100)
    val_dataset = DummyDataset(num_samples=20)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        num_workers=0,  # Set to 0 for Colab
        collate_fn=lambda x: tuple(zip(*x))
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=0,
        collate_fn=lambda x: tuple(zip(*x))
    )

    print(f"✓ Train dataset: {len(train_dataset)} samples")
    print(f"✓ Val dataset: {len(val_dataset)} samples")
    print(f"✓ Train batches: {len(train_loader)}")
    print(f"✓ Val batches: {len(val_loader)}")

    # Test one batch
    images, targets = next(iter(train_loader))
    print(f"\nSample batch:")
    print(f"  Number of images: {len(images)}")
    print(f"  Image shape: {images[0].shape}")
    print(f"  Target keys: {targets[0].keys()}")
    print(f"  Number of boxes in first image: {len(targets[0]['boxes'])}")

    print("="*70)

    return train_loader, val_loader

In [ ]:

# ============================================================================
# RUN TRAINING (QUICK TEST VERSION)
# ============================================================================

if __name__ == "__main__":
    print("\n" + "="*70)
    print("GPU-ACCELERATED OBJECT DETECTION - TRAINING")
    print("="*70)

    # Setup configuration
    config = TrainingConfig()
    config.NUM_EPOCHS = 3  # Reduced for quick testing
    config.BATCH_SIZE = 2  # Smaller batch for testing

    # Test data loaders
    train_loader, val_loader = test_dataloader()

    # Create model
    print("\nCreating model...")
    model = create_model(num_classes=config.NUM_CLASSES, pretrained=True)
    model = model.to(config.DEVICE)

    print(f"✓ Model created and moved to {config.DEVICE}")

    # Start training
    print("\nStarting training loop...")
    print("(This is a quick test with dummy data)")
    print("Replace DummyDataset with actual COCO dataset for real training\n")

    history = train_model(model, train_loader, val_loader, config)

    print("\n✓ Training script test complete!")
    print("Next: Replace dummy data with actual COCO dataset")


GPU-ACCELERATED OBJECT DETECTION - TRAINING

TESTING DATA LOADER
Creating data loaders...
✓ Train dataset: 100 samples
✓ Val dataset: 20 samples
✓ Train batches: 25
✓ Val batches: 5

Sample batch:
  Number of images: 4
  Image shape: torch.Size([3, 640, 640])
  Target keys: dict_keys(['boxes', 'labels', 'image_id'])
  Number of boxes in first image: 7

Creating model...
✓ Model created and moved to cuda

Starting training loop...
(This is a quick test with dummy data)
Replace DummyDataset with actual COCO dataset for real training


STARTING TRAINING

Epoch 1/3
Learning Rate: 0.005000


Epoch 1:  40%|████      | 10/25 [00:07<00:10,  1.38it/s, loss=2.3813, batch_time=0.782s, data_time=0.034s]


Epoch: [1][10/25]  Loss: 2.3813  Batch Time: 0.782s  Data Time: 0.034s
  loss_classifier: 0.1446
  loss_box_reg: 0.0352
  loss_objectness: 0.2336
  loss_rpn_box_reg: 0.0401


Epoch 1:  80%|████████  | 20/25 [00:15<00:03,  1.38it/s, loss=1.7003, batch_time=0.755s, data_time=0.037s]


Epoch: [1][20/25]  Loss: 1.7003  Batch Time: 0.755s  Data Time: 0.037s
  loss_classifier: 0.1984
  loss_box_reg: 0.0627
  loss_objectness: 0.3151
  loss_rpn_box_reg: 0.2702


...


Epoch: [3][20/25]  Loss: 0.9037  Batch Time: 0.752s  Data Time: 0.037s
  loss_classifier: 0.2653
  loss_box_reg: 0.1151
  loss_objectness: 0.1390
  loss_rpn_box_reg: 0.0648


Epoch 3: 100%|██████████| 25/25 [00:18<00:00,  1.33it/s, loss=0.8959, batch_time=0.755s, data_time=0.039s]



Running validation...


Validation: 100%|██████████| 5/5 [00:01<00:00,  2.85it/s, loss=1.0758, inference_time=0.315s]


Epoch 3 Summary:
  Train Loss: 0.8959
  Val Loss: 1.0758
  Val Inference Time: 0.315s
  Epoch Time: 20.64s

TRAINING COMPLETE!
Total Training Time: 0.02 hours
Best Validation Loss: 0.9027
✓ Training history saved: logs/training_history_20251005_151905.json

✓ Training script test complete!
Next: Replace dummy data with actual COCO dataset
